In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import lxml
import requests
from PIL import Image 
import io
import re
import os
import urllib
from urllib.request import Request, urlopen, HTTPSHandler
import time
import requests
import shutil


In [27]:
def scrape_from_link(link, ms_name, page, session, payload):
    print('starting to scrape page {}...'.format(page))
    tries = 3
    i = 0
    while i<3:
        try:
            s = session.get(link)
#             soup = BeautifulSoup(open(html_path, 'r'),"html.parser",from_encoding="iso-8859-1")
            soup = BeautifulSoup(s.content, 'html.parser', from_encoding="iso-8859-1")
            path = 'C:\\Users\\User\\PycharmProjects\\scrapers\\{}\\{}.jpg'.format(ms_name,page)
            if s.status_code == 200:
                print('entered status 200')
                s.raw.decode_content = True
                with open(path,'wb') as f:
                    for chunk in s:
                        f.write(chunk)
                break
            else:
                print('status problem: {}'.format(s.status_code))
                i= i+1
            
        except:
            print('status problem: {}'.format(s.status_code))
            s = session.post("https://www.vhmml.org/j_spring_security_check", data=payload)
            print('retry..')
            i= i+1 

In [34]:
#vhmml format of images
def scrape_vhmml_jpgs(ms_name, first_page, last_page, pattern):
    
    os.makedirs(ms_name, exist_ok=True)
    session = requests.Session()
    payload = {'j_username':'maeirnoam@gmail.com', 
              'j_password':'Kids1120'
             }
    s = session.post("https://www.vhmml.org/j_spring_security_check", data=payload)
    
    if pattern=='single':
        last = re.search(single_page_reg, last_page)
        first = re.search(single_page_reg, first_page)
        single_pages = [f"{i:03d}" for i in range(int(first.group(2)), int(last.group(2))+1)]
        scrape_from_link(first_page, ms_name, f'{int(first.group(2))}', session, payload)
        for i in range(len(single_pages)-2):
            link = '{}{}{}'.format(first.group(1), single_pages[i+1],first.group(4))
            scrape_from_link(link, ms_name, f'{int(first.group(2))+i+1}', session, payload)

            
#         if first.group(3)=='r':
#             link = '{}{}{}{}'.format(first.group(1), first.group(2),
#                                       'v', first.group(4))
#             scrape_from_link(link, ms_name, f'{int(first.group(2))} v', session, payload)
#         for i in range(len(single_pages)-2):
#             link_r = '{}{}{}{}'.format(first.group(1), single_pages[i+1],
#                                       'r', first.group(4))
#             scrape_from_link(link_r, ms_name, f'{int(first.group(2))+i+1} r', session, payload)
#             link_v = '{}{}{}{}'.format(first.group(1), single_pages[i+1],
#                                       'v', first.group(4))
#             scrape_from_link(link_v, ms_name, f'{int(first.group(2))+i+1} v', session, payload)
#         if last.group(3)=='v':
#             link = '{}{}{}{}'.format(first.group(1), last.group(2),
#                                       'r', first.group(4))
#             scrape_from_link(link, ms_name, f'{int(last.group(2))} r', session, payload)
        
        scrape_from_link(last_page, ms_name, f'{int(last.group(2))}', session, payload)
    if pattern=='double':
        last = re.search(double_page_reg, last_page)
        first = re.search(double_page_reg, first_page)
        double_pages = [f"{i:03d}" for i in range(int(first.group(4)), int(last.group(2))+1)]
        first_page = first.group(4)
        for i in range(len(double_pages)-1):
            link = '{}_{}{}{}{}{}'.format(first.group(1), double_pages[i+1], first.group(3),
                                         double_pages[i], first.group(5), first.group(6))
            scrape_from_link(link, ms_name, f'{int(first.group(2))+i+1}', session, payload)

In [28]:
ms_name= 'SMMJ_00180'
last_page = 'https://www.vhmml.org/image/READING_ROOM/SMMJ/SMMJ%2000180/SMMJ_00180__102.JPG/full/1000,/0/default.jpg'
first_page = 'https://www.vhmml.org/image/READING_ROOM/SMMJ/SMMJ%2000180/SMMJ_00180__011.JPG/full/1000,/0/default.jpg'
first_part = 'https://www.vhmml.org/image/READING_ROOM/'
second_part = 'SMMJ/SMMJ%2000180/SMMJ_00180'
full_link = f'{first_part}{second_part}'

# os.makedirs(ms_name, exist_ok=True)
# session = requests.Session()
# payload = {'j_username':'maeirnoam@gmail.com', 
#           'j_password':'Kids1120'
#          }
#s = session.post("https://www.vhmml.org/j_spring_security_check", data=payload)

#scrape_from_link(first_page, ms_name, 11, session, payload)

starting to scrape page 11...
entered status 200


In [ ]:
single_pages = [f"{i:03d}" for i in range(int(first.group(2)), int(last.group(2))+1)]
for i in range(len(single_pages)-2):
    link = '{}{}{}{}'.format(first.group(1), single_pages[i+1],first.group(4))
    scrape_from_link(link_r, ms_name, f'{int(first.group(2))+i+1}', session, payload)

In [32]:
ms_name= 'SMMJ_00180'
last_page = 'https://www.vhmml.org/image/READING_ROOM/SMMJ/SMMJ%2000180/SMMJ_00180__102.JPG/full/1000,/0/default.jpg'
first_page = 'https://www.vhmml.org/image/READING_ROOM/SMMJ/SMMJ%2000180/SMMJ_00180__011.JPG/full/1000,/0/default.jpg'
first_part = 'https://www.vhmml.org/image/READING_ROOM/'
second_part = 'SMMJ/SMMJ%2000180/SMMJ_00180'
full_link = f'{first_part}{second_part}'

double_page_reg = re.compile('(.*)_(\d+?)(r_)(\d+?)(v\..+)(.jpg)',
            re.DOTALL | re.MULTILINE | re.UNICODE)
single_page_reg = re.compile(
        r'(https://www.vhmml.org/image/READING_ROOM/SMMJ/SMMJ%2000180/SMMJ_00180__)(\d+)([rv]?)(\..+jpg)',
        re.DOTALL | re.MULTILINE | re.UNICODE)

#last_page = 'https://www.vhmml.org/image/READING_ROOM/CET/CET%2000076/CET_00076_354v.JPG/full/1000,/0/default.jpg'
#first_page = 'https://www.vhmml.org/image/READING_ROOM/CET/CET%2000076/CET_00076_092r.JPG/full/1000,/0/default.jpg'


In [35]:
#this is working, run from here
scrape_vhmml_jpgs(ms_name, first_page, last_page, pattern='single')

starting to scrape page 11...
entered status 200
starting to scrape page 12...
entered status 200
starting to scrape page 13...
entered status 200
starting to scrape page 14...
entered status 200
starting to scrape page 15...
entered status 200
starting to scrape page 16...
entered status 200
starting to scrape page 17...
entered status 200
starting to scrape page 18...
entered status 200
starting to scrape page 19...
entered status 200
starting to scrape page 20...
entered status 200
starting to scrape page 21...
entered status 200
starting to scrape page 22...
entered status 200
starting to scrape page 23...
entered status 200
starting to scrape page 24...
entered status 200
starting to scrape page 25...
entered status 200
starting to scrape page 26...
entered status 200
starting to scrape page 27...
entered status 200
starting to scrape page 28...
entered status 200
starting to scrape page 29...
entered status 200
starting to scrape page 30...
entered status 200
starting to scrape p

In [ ]:
first = 'https://iiif.sinaimanuscripts.library.ucla.edu/iiif/2/ark%3A%2F21198%2Fz1wd5j69%2F5x91tv66/full/1000,/0/default.jpg'
manifest = 'https://iiif.library.ucla.edu/ark%3A%2F21198%2Fz1wd5j69/manifest'
# get all id's according to '@type":"dctypes:Image'